<a href="https://colab.research.google.com/github/showrin20/MangoScan/blob/main/MangoScan_Real_Time_Mango_Species_Detection_Syste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from glob import glob
import random

import warnings
warnings.filterwarnings('ignore')


In [ ]:
dataset_path = "/kaggle/input/mango-dataset-2012"
all_classes = os.listdir(dataset_path)

print(f"✅ Total Mango Species: {len(all_classes)}")
print("📦 Classes:", all_classes)


In [ ]:
for cls in all_classes:
    files = glob(os.path.join(dataset_path, cls, "*.jpg"))
    print(f"{cls}: {len(files)} images")


In [ ]:
class_counts = {cls: len(glob(os.path.join(dataset_path, cls, "*.jpg"))) for cls in all_classes}
class_df = pd.DataFrame.from_dict(class_counts, orient='index', columns=['Image Count'])
class_df.sort_values('Image Count', ascending=False).plot(kind='bar', figsize=(10, 5), color='orange', legend=False)
plt.title("📊 Mango Species Image Distribution")
plt.ylabel("Image Count")
plt.xlabel("Species")
plt.grid(axis='y')
plt.tight_layout()
plt.show()


In [ ]:
def plot_random_images_per_class(dataset_path, classes, num_images=3):
    fig, axes = plt.subplots(len(classes), num_images, figsize=(num_images*3, len(classes)*3))

    for i, cls in enumerate(classes):
        cls_path = os.path.join(dataset_path, cls)
        images = random.sample(glob(os.path.join(cls_path, "*.jpg")), num_images)

        for j in range(num_images):
            img = Image.open(images[j])
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 1:
                axes[i, j].set_title(cls, fontsize=10)

    plt.tight_layout()
    plt.show()

plot_random_images_per_class(dataset_path, all_classes[:5])  # Limit to first 5 species for display


In [ ]:
image_shapes = []

for cls in all_classes:
    cls_path = os.path.join(dataset_path, cls)
    img_paths = glob(os.path.join(cls_path, "*.jpg"))

    for img_path in random.sample(img_paths, min(20, len(img_paths))):  # Limit per class
        img = Image.open(img_path)
        image_shapes.append(img.size)

shape_df = pd.DataFrame(image_shapes, columns=['Width', 'Height'])
print("🧾 Unique Resolutions:\n", shape_df.value_counts())

plt.figure(figsize=(6, 6))
sns.histplot(shape_df['Width'], color='skyblue', label='Width', kde=True)
sns.histplot(shape_df['Height'], color='orange', label='Height', kde=True)
plt.title("📐 Image Dimension Distribution")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
def avg_rgb(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return np.mean(img, axis=(0,1))  # RGB mean

rgb_means = []
labels = []

for cls in all_classes[:5]:  # Limit for speed
    paths = glob(os.path.join(dataset_path, cls, "*.jpg"))
    for img_path in random.sample(paths, min(10, len(paths))):
        rgb_means.append(avg_rgb(img_path))
        labels.append(cls)

rgb_df = pd.DataFrame(rgb_means, columns=['R', 'G', 'B'])
rgb_df['Class'] = labels

plt.figure(figsize=(8, 6))
sns.scatterplot(data=rgb_df, x='R', y='G', hue='Class', palette='Set2', alpha=0.7)
plt.title("🎨 RGB Space Distribution (Sampled)")
plt.show()


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

DATA_DIR = '/kaggle/input/mango-dataset-2012'
BATCH_SIZE = 32
NUM_WORKERS = 2  # Kaggle allows 2 workers safely


In [ ]:
# Training transforms with augmentation
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Validation/test transforms without augmentation
val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transforms)
print(f"Total images found: {len(full_dataset)}")
print("Classes found:", full_dataset.classes)
print("Class to index mapping:", full_dataset.class_to_idx)


In [ ]:
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Override transforms for val and test subsets (no augmentation)
val_dataset.dataset.transform = val_test_transforms
test_dataset.dataset.transform = val_test_transforms

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img_tensor):
    img = img_tensor.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)
print(f"Batch size: {images.size(0)}")
print(f"Labels: {labels}")

imshow(images[0])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import timm
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
num_classes = 10  # Based on 10 mango varieties
num_epochs = 10   # Adjust as needed
batch_size = 32   # Adjust based on memory constraints

# Define transforms
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Models expect 224x224 input
    transforms.RandomHorizontalFlip(),  # Augmentation for training
    transforms.RandomRotation(10),      # Augmentation for training
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load full dataset
dataset_path = '/kaggle/input/mango-dataset-2012'
full_dataset = ImageFolder(root=dataset_path, transform=train_transforms)

# Split dataset
total_size = len(full_dataset)
train_size = int(0.7 * total_size)  # 1408 images
val_size = int(0.15 * total_size)  # 301 images
test_size = total_size - train_size - val_size  # 303 images
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Override transforms for val and test subsets (no augmentation)
val_dataset.dataset.transform = val_test_transforms
test_dataset.dataset.transform = val_test_transforms

# Verify dataset sizes and classes
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")
print(f"Classes: {full_dataset.classes}")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Model definitions
def get_mobilenet_v2(num_classes):
    model = models.mobilenet_v2(weights='DEFAULT')
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    for param in model.features.parameters():
        param.requires_grad = False
    return model.to(device)

def get_efficientnet_lite0(num_classes):
    model = timm.create_model('efficientnet_lite0', pretrained=True)
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    for name, param in model.named_parameters():
        if "classifier" not in name:
            param.requires_grad = False
    return model.to(device)

def get_squeezenet(num_classes):
    model = models.squeezenet1_1(weights='DEFAULT')
    model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1))
    model.num_classes = num_classes
    for param in model.features.parameters():
        param.requires_grad = False
    return model.to(device)

def get_shufflenet_v2(num_classes):
    model = models.shufflenet_v2_x1_0(weights='DEFAULT')
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    for param in model.conv1.parameters():
        param.requires_grad = False
    for param in model.maxpool.parameters():
        param.requires_grad = False
    for param in model.stage2.parameters():
        param.requires_grad = False
    for param in model.stage3.parameters():
        param.requires_grad = False
    for param in model.stage4.parameters():
        param.requires_grad = False
    return model.to(device)

# Dictionary of models to train
models_dict = {
    "mobilenet_v2": get_mobilenet_v2(num_classes),
    "efficientnet_lite0": get_efficientnet_lite0(num_classes),
    "squeezenet": get_squeezenet(num_classes),
    "shufflenet_v2": get_shufflenet_v2(num_classes),
}

criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, optimizer, dataloader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for inputs, labels in tqdm(dataloader, desc="Training", leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        # SqueezeNet outputs are [batch, classes, 1, 1]
        if outputs.dim() == 4:
            outputs = outputs.squeeze(-1).squeeze(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total if total > 0 else 0
    epoch_acc = correct / total if total > 0 else 0
    return epoch_loss, epoch_acc

def validate(model, dataloader):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Validation", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            if outputs.dim() == 4:
                outputs = outputs.squeeze(-1).squeeze(-1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total if total > 0 else 0
    epoch_acc = correct / total if total > 0 else 0
    return epoch_loss, epoch_acc

def test_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Testing", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            if outputs.dim() == 4:
                outputs = outputs.squeeze(-1).squeeze(-1)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total if total > 0 else 0
    return test_acc

# Training loop for all models
for model_name, model in models_dict.items():
    print(f"\n--- Training {model_name} ---")

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

    best_val_acc = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        train_loss, train_acc = train_one_epoch(model, optimizer, train_loader)
        val_loss, val_acc = validate(model, val_loader)

        print(f"Train loss: {train_loss:.4f}, Train acc: {train_acc:.4f}")
        print(f"Val   loss: {val_loss:.4f}, Val   acc: {val_acc:.4f}")

        # Save best model weights based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = model.state_dict()

    # Save best model to disk
    torch.save(best_model_wts, f"{model_name}_best.pth")
    print(f"Best val accuracy for {model_name}: {best_val_acc:.4f}")
    print(f"Saved best model weights for {model_name}.")

    # Evaluate on test set
    model.load_state_dict(best_model_wts)  # Load best weights
    test_acc = test_model(model, test_loader)
    print(f"Test accuracy for {model_name}: {test_acc:.4f}\n")